In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Cleaning and Preprocessing by ACORN Class

In [22]:
# Save the Group Letter Name to be processed here

groupletter = 'B'

In [8]:
# Import Summary Datasheet

df_summary = pd.read_csv('../raw_data/summary_v1_selection.csv')
df_summary = df_summary[['LCLid', 'start_date', 'end_date', 'number_of_days', 'data_points', 'missing_hh', 'zeros', 'tariff', 'Block', 'Acorn', 'Group', 'Classification', 'Select']]

In [6]:
df_summary[(df_summary['Acorn'] == 'ACORN-B') & (df_summary['tariff'] == 'Std') & (df_summary['missing_hh'] <= 48)]

,LCLid,start_date,end_date,number_of_days,data_points,missing_hh,zeros,tariff,Block,Acorn,Group,Classification,Select
150,MAC000606,12/04/2012 10:30,18/02/2013 00:00,311,14956,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,False
172,MAC003355,26/09/2012 11:30,28/02/2014 00:00,519,24938,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
176,MAC003403,28/09/2012 13:00,28/02/2014 00:00,517,24837,2,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
179,MAC003445,02/10/2012 10:30,28/02/2014 00:00,513,24652,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
183,MAC003822,23/10/2012 08:30,28/02/2014 00:00,492,23644,4,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
184,MAC003876,25/10/2012 12:30,28/02/2014 00:00,490,23544,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
190,MAC004417,19/03/2012 12:30,28/02/2014 00:00,710,34098,6,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
192,MAC004567,23/12/2011 09:00,28/02/2014 00:00,797,38272,15,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
197,MAC004957,23/01/2012 11:30,28/02/2014 00:00,766,36792,2,1074,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,False


In [23]:
# Reduce selection to chosen group which has been selected
string = f'ACORN-{groupletter}'
df_hot = df_summary[(df_summary['Acorn'] == string) 
                    & (df_summary['number_of_days'] >= 365) 
                    & (df_summary['zeros'] + df_summary['missing_hh'] <= 48)]
df_hot

,LCLid,start_date,end_date,number_of_days,data_points,missing_hh,zeros,tariff,Block,Acorn,Group,Classification,Select
152,MAC001410,18/05/2012 07:30,28/02/2014 00:00,650,31231,3,2,ToU,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
172,MAC003355,26/09/2012 11:30,28/02/2014 00:00,519,24938,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
176,MAC003403,28/09/2012 13:00,28/02/2014 00:00,517,24837,2,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
179,MAC003445,02/10/2012 10:30,28/02/2014 00:00,513,24652,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
183,MAC003822,23/10/2012 08:30,28/02/2014 00:00,492,23644,4,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
184,MAC003876,25/10/2012 12:30,28/02/2014 00:00,490,23544,0,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
186,MAC003903,26/10/2012 10:00,28/02/2014 00:00,489,23498,3,0,ToU,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
190,MAC004417,19/03/2012 12:30,28/02/2014 00:00,710,34098,6,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True
192,MAC004567,23/12/2011 09:00,28/02/2014 00:00,797,38272,15,0,Std,block_3,ACORN-B,Executive Wealth,Affluent Achievers,True


In [24]:
# Create a list of houses matching criteria

houselist = df_hot['LCLid'].tolist()

In [25]:
# Create a second list of the blocks where the data for each house is found

blocklist = df_hot['Block'].tolist()

In [26]:
# Create a third list of the tariff type

tarifflist = df_hot['tariff'].tolist()

In [57]:
df_house = pd.read_csv(f'../raw_data/halfhourly_dataset/block_3.csv')
df_house = df_house[df_house['LCLid'] == houselist[3]]
df_house['tariff'] = 'Std'
df_house

,LCLid,tstp,energy(kWh/hh),tariff
804212,MAC003445,2012-10-02 10:30:00.0000000,0.226,Std
804213,MAC003445,2012-10-02 11:00:00.0000000,0.343,Std
804214,MAC003445,2012-10-02 11:30:00.0000000,0.736,Std
804215,MAC003445,2012-10-02 12:00:00.0000000,0.569,Std
804216,MAC003445,2012-10-02 12:30:00.0000000,0.754,Std
...,...,...,...,...
828860,MAC003445,2014-02-27 22:00:00.0000000,0.476,Std
828861,MAC003445,2014-02-27 22:30:00.0000000,0.307,Std
828862,MAC003445,2014-02-27 23:00:00.0000000,0.171,Std
828863,MAC003445,2014-02-27 23:30:00.0000000,0.136,Std


In [68]:
# Iterable loop concactinating the data into lists
pd.options.mode.chained_assignment = None  # default='warn'
LCL = []
Acorn_Group = []
DateTime = []
KWH = []
tariff = []

df_house = pd.read_csv(f'../raw_data/halfhourly_dataset/block_3.csv')
for i in range (0,int(len(houselist))):
    print(f'Now adding house {i+1} of {len(houselist)}, {houselist[i]} to the dataset...')
    blockstr = f'../raw_data/halfhourly_dataset/{blocklist[i]}.csv'
    if blocklist[i] != blocklist[i-1]:
        df_house = pd.read_csv(blockstr, dtype = {'LCLid': object, 'tstp': object , 'energy(kWh/hh)': object})
    df2_house = df_house[df_house['LCLid'] == houselist[i]]
    df2_house['tariff'] = tarifflist[i]
    LCL.extend(df2_house['LCLid'])
    DateTime.extend(df2_house['tstp'])
    KWH.extend(df2_house['energy(kWh/hh)'])
    tariff.extend(df2_house['tariff'])
    

Now adding house 1 of 9, MAC001410 to the dataset...
Now adding house 2 of 9, MAC003355 to the dataset...
Now adding house 3 of 9, MAC003403 to the dataset...
Now adding house 4 of 9, MAC003445 to the dataset...
Now adding house 5 of 9, MAC003822 to the dataset...
Now adding house 6 of 9, MAC003876 to the dataset...
Now adding house 7 of 9, MAC003903 to the dataset...
Now adding house 8 of 9, MAC004417 to the dataset...
Now adding house 9 of 9, MAC004567 to the dataset...


In [69]:
df2_house

,LCLid,tstp,energy(kWh/hh),tariff
1133590,MAC004567,2011-12-23 09:00:00.0000000,0.102,Std
1133591,MAC004567,2011-12-23 09:30:00.0000000,0.097,Std
1133592,MAC004567,2011-12-23 10:00:00.0000000,0.176,Std
1133593,MAC004567,2011-12-23 10:30:00.0000000,0.14,Std
1133594,MAC004567,2011-12-23 11:00:00.0000000,0.166,Std
...,...,...,...,...
1171858,MAC004567,2014-02-27 22:00:00.0000000,0.243,Std
1171859,MAC004567,2014-02-27 22:30:00.0000000,0.724,Std
1171860,MAC004567,2014-02-27 23:00:00.0000000,0.163,Std
1171861,MAC004567,2014-02-27 23:30:00.0000000,0.05,Std


In [67]:
set(tariff)

{'Std', 'ToU'}

In [30]:
# Creating a new Data Frame with lists

block_data = pd.DataFrame({'LCLid': LCL, 'DateTime': DateTime, 'KWH/hh': KWH, 'Tariff': tariff})

In [31]:
# Remove Null values

block_data = block_data[block_data['KWH/hh'] != 'Null']

# Convert KWH/hh to numeric

block_data['KWH/hh'] = pd.to_numeric(block_data['KWH/hh'])

# Convert DateTime to DateTime format

block_data['DateTime'] = pd.to_datetime(block_data['DateTime'])

# Create ACORN group label

block_data['Acorn_Group'] = groupletter
block_data['Group'] = df_hot['Group'].iloc[0]
block_data['Classification'] = df_hot['Classification'].iloc[0]

In [54]:
block_data

,LCLid,DateTime,KWH/hh,Tariff,Acorn_Group,Group,Classification
0,MAC000246,2011-12-03 09:00:00,0.149,Std,A,Lavish Lifestyles,Affluent Achievers
1,MAC000246,2011-12-03 09:30:00,0.154,Std,A,Lavish Lifestyles,Affluent Achievers
2,MAC000246,2011-12-03 10:00:00,0.768,Std,A,Lavish Lifestyles,Affluent Achievers
3,MAC000246,2011-12-03 10:30:00,1.179,Std,A,Lavish Lifestyles,Affluent Achievers
4,MAC000246,2011-12-03 11:00:00,0.588,Std,A,Lavish Lifestyles,Affluent Achievers
...,...,...,...,...,...,...,...
2365795,MAC005159,2014-02-27 22:00:00,0.424,ToU,A,Lavish Lifestyles,Affluent Achievers
2365796,MAC005159,2014-02-27 22:30:00,0.388,ToU,A,Lavish Lifestyles,Affluent Achievers
2365797,MAC005159,2014-02-27 23:00:00,0.276,ToU,A,Lavish Lifestyles,Affluent Achievers
2365798,MAC005159,2014-02-27 23:30:00,0.255,ToU,A,Lavish Lifestyles,Affluent Achievers


In [32]:
block_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31231 entries, 0 to 31231
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   LCLid           31231 non-null  object        
 1   DateTime        31231 non-null  datetime64[ns]
 2   KWH/hh          31231 non-null  float64       
 3   Tariff          31231 non-null  object        
 4   Acorn_Group     31231 non-null  object        
 5   Group           31231 non-null  object        
 6   Classification  31231 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.9+ MB


In [33]:
block_data

,LCLid,DateTime,KWH/hh,Tariff,Acorn_Group,Group,Classification
0,MAC001410,2012-05-18 07:30:00,0.000,ToU,B,Executive Wealth,Affluent Achievers
1,MAC001410,2012-05-18 08:00:00,0.000,ToU,B,Executive Wealth,Affluent Achievers
2,MAC001410,2012-05-18 09:00:00,0.138,ToU,B,Executive Wealth,Affluent Achievers
3,MAC001410,2012-05-18 09:30:00,0.144,ToU,B,Executive Wealth,Affluent Achievers
4,MAC001410,2012-05-18 10:00:00,0.145,ToU,B,Executive Wealth,Affluent Achievers
...,...,...,...,...,...,...,...
31227,MAC001410,2014-02-27 22:00:00,0.282,ToU,B,Executive Wealth,Affluent Achievers
31228,MAC001410,2014-02-27 22:30:00,0.261,ToU,B,Executive Wealth,Affluent Achievers
31229,MAC001410,2014-02-27 23:00:00,0.256,ToU,B,Executive Wealth,Affluent Achievers
31230,MAC001410,2014-02-27 23:30:00,0.287,ToU,B,Executive Wealth,Affluent Achievers


In [34]:
# Fill 0 with nans
block_data['KWH/hh'].replace(0,np.nan,inplace=True)

# Fill nan's using a back fill
date_range = pd.DataFrame(pd.date_range(block_data.index[0],block_data.index[-1], freq='30 min'),columns=['DateTime'])
block_data = date_range.merge(block_data,on='DateTime',how='outer')
if np.sum(block_data['KWH/hh'].isna())!=0:
    block_data.fillna(method='bfill',inplace=True)


In [35]:
block_data = block_data[block_data['DateTime'] >= '2012-01-01']

In [36]:
# Identify and remove outliers

u = np.mean(block_data['KWH/hh'])
q3 = np.quantile(block_data['KWH/hh'], 0.75)
q1 = np.quantile(block_data['KWH/hh'], 0.25)
IQR = q3-q1
ub = q3 + 1.5 * IQR
lb = q1 - 1.5 * IQR
block_data[(block_data['KWH/hh'] <= ub) & (block_data['KWH/hh'] >= lb)]

,DateTime,LCLid,KWH/hh,Tariff,Acorn_Group,Group,Classification
1,2012-05-18 07:30:00,MAC001410,0.138,ToU,B,Executive Wealth,Affluent Achievers
2,2012-05-18 08:00:00,MAC001410,0.138,ToU,B,Executive Wealth,Affluent Achievers
3,2012-05-18 09:00:00,MAC001410,0.138,ToU,B,Executive Wealth,Affluent Achievers
4,2012-05-18 09:30:00,MAC001410,0.144,ToU,B,Executive Wealth,Affluent Achievers
5,2012-05-18 10:00:00,MAC001410,0.145,ToU,B,Executive Wealth,Affluent Achievers
...,...,...,...,...,...,...,...
31227,2014-02-27 22:00:00,MAC001410,0.282,ToU,B,Executive Wealth,Affluent Achievers
31228,2014-02-27 22:30:00,MAC001410,0.261,ToU,B,Executive Wealth,Affluent Achievers
31229,2014-02-27 23:00:00,MAC001410,0.256,ToU,B,Executive Wealth,Affluent Achievers
31230,2014-02-27 23:30:00,MAC001410,0.287,ToU,B,Executive Wealth,Affluent Achievers


In [37]:
# Create average time
block_data_avg=block_data.groupby(by=block_data.DateTime).mean()
block_data_avg.reset_index(inplace = True)

In [38]:
# View the final dataset

block_data_avg

,DateTime,KWH/hh
0,2012-05-18 07:30:00,0.138
1,2012-05-18 08:00:00,0.138
2,2012-05-18 09:00:00,0.138
3,2012-05-18 09:30:00,0.144
4,2012-05-18 10:00:00,0.145
...,...,...
31226,2014-02-27 22:00:00,0.282
31227,2014-02-27 22:30:00,0.261
31228,2014-02-27 23:00:00,0.256
31229,2014-02-27 23:30:00,0.287


In [69]:
# Save and Export dataset to CSV
filename = f'df_{groupletter}__avg_v1.csv'

block_data_avg.to_csv(f'../raw_data/{filename}')

In [71]:
block_data['LCLid'].nunique()

81